In [1]:
import os
os.environ['HANLP_HOME'] = '/dlabscratch1/tdai/hanlp'
import hanlp
import chinese_converter
import random
import re
from tqdm import tqdm
import math
from collections import defaultdict
import hanlp.utils
import pandas as pd
from collections import Counter
from gensim.models import Word2Vec
import pandas as pd
from matplotlib import font_manager
import matplotlib.pyplot as plt
from matplotlib import rcParams

plt.rcParams["font.sans-serif"] = ["SimHei"]  # 用来正常显示中文标签
plt.rcParams["axes.unicode_minus"] = False  # 用来正常显示负号
print("SimHei" in [f.name for f in font_manager.fontManager.ttflist])  

True


In [11]:
book = {}
book_file_name = '朱天心-1996-古都.txt'.encode('utf-8').decode('latin1')
with open(f'/dlabscratch1/tdai/history/book/{book_file_name}', 'r', encoding='utf-8') as f:
    book['古都'] = chinese_converter.to_simplified(f.read())

book_file_name = '朱天心-1977-擊壤歌.txt'.encode('utf-8').decode('latin1')
with open(f'/dlabscratch1/tdai/history/book/{book_file_name}', 'r', encoding='utf-8') as f:
    book['击壤歌'] = chinese_converter.to_simplified(f.read())

book

{'古都': '※我在圣马可广场，看到天使飞翔的特技，摩尔人跳舞，但没有妳，亲爱的，我孤独难耐。\u3000\u3000──I. V. Foscarini\n\n\n\n\u3000\u3000难道，你的记忆都不算数……\n\n\n\n\u3000\u3000那时候的天空蓝多了，蓝得让人老念着那大海就在不远处好想去，因此夏天的积乱云堡雪砌成般的显得格外白，阳光穿过未有阻拦的干净空气特强烈，奇怪并不觉其热，起码傻傻的站在无遮荫处，不知何去何从一下午，也从没半点中暑迹象。\n\n\n\n\u3000\u3000那时候的体液和泪水清新如花露，人们比较愿意随它要落就落。\n\n\n\n\u3000\u3000那时候的人们非常单纯天真，不分党派的往往为了单一的信念或爱人，肯于舍身或赴死。\n\n\n\n\u3000\u3000那时候的树，也因土地尚未商品化，没大肆开路竞建炒地皮，而得以存活得特别高大特别绿，像赤道雨林的国家。\n\n\n\n\u3000\u3000那时候鲜有公共场所，咖啡馆非常少，速食店辔沫红茶ＫＴＶ、ＰＵＢ更是不用说，少年的只好四处游荡猛走，但路上也不见人潮汹涌白老鼠一般。\n\n\n\n\u3000\u3000那时候的夏天夜晚通常都看得到银河和流星，望之久久便会生出人世存亡朝代兴衰之感，其中比较傻的就有立誓将来要做番大事绝不虚度此生。\n\n\n\n\u3000\u3000那时候的背景音乐，若你有个念大学的哥哥或姊姊，你可能多少还在听披头四。要是七○年代的第一年，那么不分时地得听Candida，以及第二年同一个合唱团的敲三下，若是六九年末，你就一定听过Aquarius，电视节目《欢乐宫》里每播三次准会出现一次的那个黑人合唱团The 5th Dimension。再早一点的话，你一定听过学士合唱团的〈Can`t take my eyes off you〉，错过这首的人，十年之后可以再在《越战猎鹿人》里的那场酒吧戏听到。\n\n\n\n\u3000\u3000虽然你喜欢的是Don McLean的〈Vincent〉和〈American Pie〉，为此我们只好把时间延后两年──且让我确定一下资料，《Vincent》是七二年五月十三日登上排行榜，那么，这就是七二年的夏天吧，你充耳不闻舞会里的热场第一名三犬夜的〈Joy to the world〉，自然也不理夏天过后三犬

In [12]:
for key, value in book.items():
    text_chunks = value.replace("\u3000", "").split("\n")
    text_chunks = [chunk for chunk in text_chunks if chunk.strip()]
    text_chunks = [chunk for text_chunk in text_chunks for chunk in re.split(r'[，；。]+', text_chunk) if chunk.strip()]
    text_chunks = [chunk for chunk in text_chunks if re.search(r'[\u4e00-\u9fff]', chunk)]
    book[key] = text_chunks

book

{'古都': ['※我在圣马可广场',
  '看到天使飞翔的特技',
  '摩尔人跳舞',
  '但没有妳',
  '亲爱的',
  '我孤独难耐',
  '难道',
  '你的记忆都不算数……',
  '那时候的天空蓝多了',
  '蓝得让人老念着那大海就在不远处好想去',
  '因此夏天的积乱云堡雪砌成般的显得格外白',
  '阳光穿过未有阻拦的干净空气特强烈',
  '奇怪并不觉其热',
  '起码傻傻的站在无遮荫处',
  '不知何去何从一下午',
  '也从没半点中暑迹象',
  '那时候的体液和泪水清新如花露',
  '人们比较愿意随它要落就落',
  '那时候的人们非常单纯天真',
  '不分党派的往往为了单一的信念或爱人',
  '肯于舍身或赴死',
  '那时候的树',
  '也因土地尚未商品化',
  '没大肆开路竞建炒地皮',
  '而得以存活得特别高大特别绿',
  '像赤道雨林的国家',
  '那时候鲜有公共场所',
  '咖啡馆非常少',
  '速食店辔沫红茶ＫＴＶ、ＰＵＢ更是不用说',
  '少年的只好四处游荡猛走',
  '但路上也不见人潮汹涌白老鼠一般',
  '那时候的夏天夜晚通常都看得到银河和流星',
  '望之久久便会生出人世存亡朝代兴衰之感',
  '其中比较傻的就有立誓将来要做番大事绝不虚度此生',
  '那时候的背景音乐',
  '若你有个念大学的哥哥或姊姊',
  '你可能多少还在听披头四',
  '要是七○年代的第一年',
  '那么不分时地得听Candida',
  '以及第二年同一个合唱团的敲三下',
  '若是六九年末',
  '你就一定听过Aquarius',
  '电视节目《欢乐宫》里每播三次准会出现一次的那个黑人合唱团The 5th Dimension',
  '再早一点的话',
  '你一定听过学士合唱团的〈Can`t take my eyes off you〉',
  '错过这首的人',
  '十年之后可以再在《越战猎鹿人》里的那场酒吧戏听到',
  '虽然你喜欢的是Don McLean的〈Vincent〉和〈American Pie〉',
  '为此我们只好把时间延后两年──且让我确定一下资料',
  '《Vincent》是七二年五月十三日登上排行榜',
  '那么',
  '这就是七二年的夏天吧'

In [16]:
HanLP = hanlp.pipeline() \
    .append(hanlp.load('COARSE_ELECTRA_SMALL_ZH'), output_key='tok') \
    .append(hanlp.load('CTB9_POS_ELECTRA_SMALL'), output_key='pos') \
    .append(hanlp.load('CTB9_DEP_ELECTRA_SMALL', conll=0), output_key='dep', input_key='tok')\

res = defaultdict(list)

for text_sample in tqdm(book['古都']):
    # HanLP(text_sample).pretty_print()
    res_ = HanLP(text_sample)
    res['古都'].append(res_)

for text_sample in tqdm(book['击壤歌']):
    # HanLP(text_sample).pretty_print()
    res_ = HanLP(text_sample)
    res['击壤歌'].append(res_)

res

100%|██████████| 9720/9720 [07:21<00:00, 22.02it/s]


defaultdict(list,
            {'古都': [{'tok': ['※', '我', '在', '圣马可广场'],
               'pos': ['PU', 'PN', 'VV', 'NR'],
               'dep': [(3, 'punct'), (3, 'nsubj'), (0, 'root'), (3, 'dobj')]},
              {'tok': ['看到', '天使', '飞翔', '的', '特技'],
               'pos': ['VV', 'NN', 'VV', 'DEC', 'NN'],
               'dep': [(0, 'root'),
                (3, 'nsubj'),
                (5, 'rcmod'),
                (3, 'cpm'),
                (1, 'dobj')]},
              {'tok': ['摩尔', '人', '跳舞'],
               'pos': ['NN', 'NN', 'VV'],
               'dep': [(2, 'nn'), (3, 'nsubj'), (0, 'root')]},
              {'tok': ['但', '没有', '妳'],
               'pos': ['AD', 'VE', 'PN'],
               'dep': [(2, 'advmod'), (0, 'root'), (2, 'dobj')]},
              {'tok': ['亲爱的'], 'pos': ['JJ'], 'dep': [(0, 'root')]},
              {'tok': ['我', '孤独', '难耐'],
               'pos': ['PN', 'VA', 'VA'],
               'dep': [(2, 'nsubj'), (0, 'root'), (2, 'advmod')]},
              {'tok': ['难

In [24]:
for i, t in enumerate(book['古都']):
    if '耕者有其田政策下被合法掠夺过家财' in t:
        print(i)
        print(t)
        break

2472
二十年后政治正确的写作者也许不难替她们安排一两位二二八受难亲族、或耕者有其田政策下被合法掠夺过家财的、或在牯岭街买到《自由中国》或《大学杂志》并因此启蒙的


In [28]:

n_pass = 0
for i, res_ in enumerate(res['古都']):
    if i == 2472:
        print(''.join(res_['tok']))
        print(res_['dep'])
    is_passive = 0
    for tup in res_['dep']:
        if tup[1] == 'pass' or tup[1] == 'nbujpass':
            is_passive = 1
            break
    n_pass += is_passive

print(f"古都被动句密度(每100句)：{n_pass*100/len(res['古都'])}")

n_pass = 0
for res_ in res['击壤歌']:
    is_passive = 0
    for tup in res_['dep']:
        if tup[1] == 'pass' or tup[1] == 'nbujpass':
            is_passive = 1
            break
    n_pass += is_passive

print(f"击壤歌被动句密度(每100句)：{n_pass*100/len(res['击壤歌'])}")

二十年后政治正确的写作者也许不难替她们安排一两位二二八受难亲族、或耕者有其田政策下被合法掠夺过家财的、或在牯岭街买到《自由中国》或《大学杂志》并因此启蒙的
[(2, 'lobj'), (12, 'loc'), (4, 'nsubj'), (6, 'rcmod'), (4, 'cpm'), (12, 'nsubj'), (12, 'advmod'), (12, 'neg'), (12, 'advmod'), (12, 'prep'), (10, 'pobj'), (0, 'root'), (16, 'nummod'), (16, 'nn'), (16, 'amod'), (19, 'dep'), (19, 'punct'), (19, 'cc'), (26, 'nsubj'), (23, 'lccomp'), (22, 'nn'), (23, 'lobj'), (26, 'loc'), (26, 'pass'), (26, 'advmod'), (12, 'dep'), (26, 'asp'), (26, 'dobj'), (26, 'cpm'), (34, 'punct'), (34, 'cc'), (34, 'prep'), (32, 'pobj'), (12, 'dep'), (34, 'punct'), (35, 'amod'), (35, 'punct'), (35, 'cc'), (34, 'punct'), (39, 'nn'), (34, 'punct'), (44, 'advmod'), (44, 'advmod'), (12, 'dep'), (44, 'cpm')]
古都被动句密度(每100句)：1.4127764127764129
击壤歌被动句密度(每100句)：0.3292181069958848
